In [3]:
import numpy as np
import pandas as pd

In [4]:
def lattmat(df,ajdi,relaxation_step_number):
    """Create a numpy matrix of lattice vectors. A column = a vector.
    
    Keyword arguments:
    df -- dataframe of the lattice vectors
    ajdi -- id of the material
    relaxation_step_number -- self explanatory
    """
    return np.transpose(df.loc[(df["id"]==ajdi) & (df["relaxation_step_number"]==relaxation_step_number)].loc[:,["lattice_vector_1_x" ,"lattice_vector_1_y" ,"lattice_vector_1_z" ,"lattice_vector_2_x" ,"lattice_vector_2_y" ,"lattice_vector_2_z" ,"lattice_vector_3_x" ,"lattice_vector_3_y", "lattice_vector_3_z"]].to_numpy(dtype=float).reshape(3,3))

In [4]:
import time
def execution_time(func):
    def wrap(*args, **kwargs):
        start = time.time()
        executed = func(*args, **kwargs)
        end = time.time()
        print("Execution time of " + str(func) + ":", end-start, "s.")
        return executed
    return wrap

# Forces and Pressure

In [43]:
def get_forces_tensors_and_pressure(raw_data_file, ajdi, df_frac):
    with open(raw_data_file, 'r') as file:
        dic_1={}
        dic_2={}
        dummy=[]
        rsn_1 = 0
        rsn_2 = 0
        rsn_1_lst=[]
        rsn_2_lst=[]
        ajdi_1_lst=[]
        ajdi_2_lst=[]
        x=[]
        y=[]
        z=[]
        dic={}
        xx=[]
        xy=[]
        xz=[]
        yy=[]
        yz=[]
        zz=[]
        pressure=[]
        while True:
            try:
                line = next(file)
                if "Analytical stress tensor - Symmetrized" in line:
                    [next(file) for _ in range(4)]
                    dummy=[]
                    for _ in range(3):
                        line = next(file)
                        dummy.append(line.split()[2:-1])
                    ajdi_1_lst.append(ajdi)
                    rsn_1_lst.append(rsn_1)
                    xx.append(dummy[0][0])
                    xy.append(dummy[0][1])
                    xz.append(dummy[0][2])
                    yy.append(dummy[1][1])
                    yz.append(dummy[1][2])
                    zz.append(dummy[2][2])
                    next(file)
                    line = next(file)
                    pressure.append(line.split()[2:-2][0])
                    rsn_1 += 1

                if "Total atomic forces" in line:
                    line = next(file)
                    #while "  |" in line:
                    for _ in range(len(df_frac[(df_frac["id"]==ajdi) & (df_frac["relaxation_step_number"]==rsn_2)])):
                        data = line.split()[2:]
                        ajdi_2_lst.append(ajdi)
                        rsn_2_lst.append(rsn_2)
                        x.append(data[0])
                        y.append(data[1])
                        z.append(data[2])
                        line = next(file)
                    rsn_2 += 1

            except StopIteration:
                dic = {"id": df_frac.id.loc[(df_frac["id"]==ajdi)].to_list(), "relaxation_step_number": df_frac.relaxation_step_number.loc[(df_frac["id"]==ajdi)].to_list(), "species": df_frac.species.loc[(df_frac["id"]==ajdi)].to_list(), "Fx": x, "Fy": y, "Fz": z}
                df_forces = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)

                dic = {"id": ajdi_1_lst, "relaxation_step_number": rsn_1_lst, "xx": xx, "yy": yy, "zz": zz, "xy": xy, "xz": xz, "yz": yz, "pressure": pressure}
                df_stress_tensor_and_pressure = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)
                print("All tensors, pressures and forces from id", ajdi, "extracted.", rsn_1, rsn_2)
                return df_stress_tensor_and_pressure, df_forces

In [52]:
# test set

df_stress_tensor_and_pressure = pd.DataFrame(columns=['id', 'relaxation_step_number', 'xx', 'yy', 'zz', 'xy', 'xz', 'yz', 'pressure'])
df_forces = pd.DataFrame(columns=['id', 'relaxation_step_number', 'species', 'Fx', 'Fy', 'Fz'])

df_frac = pd.read_csv("/home/jurka/research-project/test/relaxation/with_all_zeros/atoms_frac_xyz_with_zeros.csv")

list_of_file_numbers = [i for i in range(1,601)]

for i in list_of_file_numbers:
    path = "".join(["/home/jurka/Desktop/REE/nomad_kaggle_bandfiles/test_bandfiles/just_bandfiles/", str(i), "/band.out"])
    
    df1, df2 = get_forces_tensors_and_pressure(path, i, df_frac)
     
    df_stress_tensor_and_pressure = pd.concat([df_stress_tensor_and_pressure, df1]).reset_index(drop=True)
    df_forces = pd.concat([df_forces, df2]).reset_index(drop=True)
        
df_stress_tensor_and_pressure.to_csv("/home/jurka/research-project/test/additional/stress_tensor_and_pressure.csv", index = False)
df_forces.to_csv("/home/jurka/research-project/test/additional/forces.csv", index = False)

All tensors, pressures and forces from id 1 extracted. 24 24
All tensors, pressures and forces from id 2 extracted. 1 1
All tensors, pressures and forces from id 3 extracted. 7 7
All tensors, pressures and forces from id 4 extracted. 14 14
All tensors, pressures and forces from id 5 extracted. 7 7
All tensors, pressures and forces from id 6 extracted. 1 1
All tensors, pressures and forces from id 7 extracted. 2 2
All tensors, pressures and forces from id 8 extracted. 22 22
All tensors, pressures and forces from id 9 extracted. 1 1
All tensors, pressures and forces from id 10 extracted. 19 18
All tensors, pressures and forces from id 11 extracted. 3 3
All tensors, pressures and forces from id 12 extracted. 2 2
All tensors, pressures and forces from id 13 extracted. 13 13
All tensors, pressures and forces from id 14 extracted. 1 1
All tensors, pressures and forces from id 15 extracted. 12 12
All tensors, pressures and forces from id 16 extracted. 9 9
All tensors, pressures and forces fro

In [54]:
# train set

df_stress_tensor_and_pressure = pd.DataFrame(columns=['id', 'relaxation_step_number', 'xx', 'yy', 'zz', 'xy', 'xz', 'yz', 'pressure'])
df_forces = pd.DataFrame(columns=['id', 'relaxation_step_number', 'species', 'Fx', 'Fy', 'Fz'])

df_frac = pd.read_csv("/home/jurka/research-project/train/relaxation/with_all_zeros/atoms_frac_xyz_with_zeros.csv")

list_of_file_numbers = [i for i in range(1,2401)]   

# Problematic datafiles:
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(464)])
del list_of_file_numbers[list_of_file_numbers.index(464)] # 464. is problematic and will not be included in the dataset
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2189)])
del list_of_file_numbers[list_of_file_numbers.index(2189)] # 2189. is problematic and will not be included in the dataset

# Removing duplicates from the train set:
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(126)])
del list_of_file_numbers[list_of_file_numbers.index(126)] # id 395/126 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(1215)])
del list_of_file_numbers[list_of_file_numbers.index(1215)] # id 1215/1886 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(353)])
del list_of_file_numbers[list_of_file_numbers.index(353)] # id 2075/353 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(308)])
del list_of_file_numbers[list_of_file_numbers.index(308)] # id 308/2154 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(531)])
del list_of_file_numbers[list_of_file_numbers.index(531)] # id 531/1379 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2319)])
del list_of_file_numbers[list_of_file_numbers.index(2319)] # id 2319/2337 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2370)])
del list_of_file_numbers[list_of_file_numbers.index(2370)] # id 2370/2333 is duplicate. One removed.

for i in list_of_file_numbers:
    path = "".join(["/home/jurka/Desktop/REE/nomad_kaggle_bandfiles/train_bandfiles/just_bandfiles/", str(i), "/band.out"])
    
    df1, df2 = get_forces_tensors_and_pressure(path, i, df_frac)
     
    df_stress_tensor_and_pressure = pd.concat([df_stress_tensor_and_pressure, df1]).reset_index(drop=True)
    df_forces = pd.concat([df_forces, df2]).reset_index(drop=True)
        
df_stress_tensor_and_pressure.to_csv("/home/jurka/research-project/train/additional/stress_tensor_and_pressure.csv", index = False)
df_forces.to_csv("/home/jurka/research-project/train/additional/forces.csv", index = False)

Removing material id 464
Removing material id 2189
Removing material id 126
Removing material id 1215
Removing material id 353
Removing material id 308
Removing material id 531
Removing material id 2319
Removing material id 2370
All tensors, pressures and forces from id 1 extracted. 1 1
All tensors, pressures and forces from id 2 extracted. 17 17
All tensors, pressures and forces from id 3 extracted. 9 8
All tensors, pressures and forces from id 4 extracted. 3 3
All tensors, pressures and forces from id 5 extracted. 7 7
All tensors, pressures and forces from id 6 extracted. 3 3
All tensors, pressures and forces from id 7 extracted. 14 12
All tensors, pressures and forces from id 8 extracted. 29 28
All tensors, pressures and forces from id 9 extracted. 19 19
All tensors, pressures and forces from id 10 extracted. 6 6
All tensors, pressures and forces from id 11 extracted. 1 1
All tensors, pressures and forces from id 12 extracted. 18 18
All tensors, pressures and forces from id 13 extra

In [ ]:
# DEPRECATED PART OF FUNCTION ZEROTH:
                if "This is the geometry file that corresponds to the current relaxation step." in line:
                    for _ in range(5):
                        next(file)
                    line = next(file)
                    x=[]
                    y=[]
                    z=[]
                    species=[]
                    dic={}
                    while "atom_frac" in line:
                        data = line.split()[-4:]
                        #print("Atomic frac:",line.split()[-4:])
                        x.append(data[0])
                        y.append(data[1])
                        z.append(data[2])
                        species.append(data[3])
                        line = next(file)
                    dic = {"id": [ajdi for _ in range(len(species))], "relaxation_step_number": [0 for _ in range(len(species))], "species": species, "L1": x, "L2": y, "L3": z}
                    df_atomic_frac = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)

# 0th materials

In [8]:
def zeroth(raw_data_file, ajdi,df_general):
    with open(raw_data_file, 'r') as file:
        lines = ""
        line = ""
        df_atomic_egy = pd.DataFrame(columns=["id","relaxation_step_number","formation_energy_ev_natom","bandgap_energy_ev"])
        while True:
            try:
                line = next(file)
                if "The structure contains" in line:
                    num = line.split()[3]
                    [next(file) for _ in range(3)]
                    dic={}
                    dummy=[]
                    for _ in range(3):
                        line = next(file)
                        lines += line
                    for line in lines.splitlines():
                        dummy.append(line.split()[1:])
                    dic = {'id': ajdi, 'relaxation_step_number': 0, 
                           'lattice_vector_1_x': float(dummy[0][0]), 
                           'lattice_vector_1_y': float(dummy[0][1]), 
                           'lattice_vector_1_z': float(dummy[0][2]),
                           'lattice_vector_2_x': float(dummy[1][0]),
                           'lattice_vector_2_y': float(dummy[1][1]),
                           'lattice_vector_2_z': float(dummy[1][2]),
                           'lattice_vector_3_x': float(dummy[2][0]),
                           'lattice_vector_3_y': float(dummy[2][1]),
                           'lattice_vector_3_z': float(dummy[2][2]),
                          }
                    df_latt = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)
                    #print("Number of atoms:", num)
                    [next(file) for _ in range(2)]
                    x=[]
                    y=[]
                    z=[]
                    species=[]
                    dic={}
                    for _ in range(int(num)):
                        line = next(file)
                        data = line.split()[-4:]
                        #print("Atomic xyz:",line.split()[-4:])
                        x.append(data[1])
                        y.append(data[2])
                        z.append(data[3])
                        species.append(data[0])
                    dic = {"id": [ajdi for _ in range(len(species))], "relaxation_step_number": [0 for _ in range(len(species))], "species": species, "x [A]": x, "y [A]": y, "z [A]": z}
                    df_atomic_xyz = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)
                
                if "Total energy                  :" in line:
                    total_energy = float(line.split()[-2])
                if "ESTIMATED overall HOMO-LUMO gap:" in line:
                    gap_energy = float(line.split()[4])
                if "Relaxation step number      1:" in line:
                    values = df_general[df_general["id"]==ajdi].values[0]
                    foo = -2.5*(float(values[3])*(-3876.638238492914) + float(values[4])*(-22502.75773920757) + float(values[5])*(-66080.69404540183) - total_energy/float(values[2]))
                    dic = {"id": ajdi, "relaxation_step_number": 0, "formation_energy_ev_natom": foo, "bandgap_energy_ev": gap_energy}
                    df_atomic_egy = pd.concat([pd.Series(v, name=k) for k, v in dic.items()], axis=1)
            except StopIteration:
                print(ajdi,"ended.")
                return df_atomic_xyz, df_latt, df_atomic_egy

In [9]:
# test set
df_general = pd.read_csv("/home/jurka/research-project/test/relaxation/general.csv")
df_egy = pd.read_csv("/home/jurka/research-project/test/relaxation/energy_relaxation.csv")
df_latt = pd.read_csv("/home/jurka/research-project/test/relaxation/lattice_vector_relaxation.csv")
df_frac = pd.read_csv("/home/jurka/research-project/test/relaxation/atoms_frac_xyz_relaxation.csv")
df_xyz = pd.read_csv("/home/jurka/research-project/test/relaxation/atoms_xyz_relaxation.csv")

list_of_file_numbers = [i for i in range(1,601)]

for i in list_of_file_numbers:
    path = "".join(["/home/jurka/Desktop/REE/nomad_kaggle_bandfiles/test_bandfiles/just_bandfiles/", str(i), "/band.out"])
    
    df_xyz_zeroth, df_latt_zeroth, df_egy_zeroth = zeroth(path, i, df_general)
    
    
    #df_xyz = pd.concat([df_xyz, df_xyz_zeroth]).reset_index(drop=True)
    #df_latt = pd.concat([df_latt, df_latt_zeroth]).reset_index(drop=True)
    
    A = np.linalg.inv(lattmat(df_latt_zeroth,i,0))
    
    material = df_xyz_zeroth.loc[df_xyz_zeroth["id"]==i][df_xyz_zeroth.columns[3:].to_list()].to_numpy(dtype=float)
    
    material_frac = np.round(np.dot(material,np.transpose(A)),decimals=6)
    
    df_frac_zeroth = pd.DataFrame(material_frac, columns=["L1", "L2", "L3"])
    df_frac_zeroth = df_xyz_zeroth[df_xyz_zeroth.columns.to_list()[:3]].join(df_frac_zeroth,how="left")
    
    if len(df_latt[(df_latt["id"]==i) & (df_latt["relaxation_step_number"]==0)])==0:
        pos = df_frac.loc[df_frac["id"]==i].iloc[0].name
        df_frac = pd.concat([df_frac.iloc[:pos], df_frac_zeroth, df_frac.iloc[pos:]]).reset_index(drop=True)

        pos = df_xyz.loc[df_xyz["id"]==i].iloc[0].name
        df_xyz = pd.concat([df_xyz.iloc[:pos], df_xyz_zeroth, df_xyz.iloc[pos:]]).reset_index(drop=True)

        pos = df_latt.loc[df_latt["id"]==i].iloc[0].name
        df_latt = pd.concat([df_latt.iloc[:pos], df_latt_zeroth, df_latt.iloc[pos:]]).reset_index(drop=True)
        
        pos = df_egy.loc[df_egy["id"]==i].iloc[0].name
        df_egy = pd.concat([df_egy.iloc[:pos], df_egy_zeroth, df_egy.iloc[pos:]]).reset_index(drop=True)
        
df_latt.to_csv("/home/jurka/research-project/test/relaxation/with_all_zeroes/lattice.csv", index = False)
df_egy.to_csv("/home/jurka/research-project/test/relaxation/with_all_zeroes/energy.csv", index = False)
df_xyz.to_csv("/home/jurka/research-project/test/relaxation/with_all_zeroes/xyz.csv", index = False)
df_frac.to_csv("/home/jurka/research-project/test/relaxation/with_all_zeroes/frac.csv", index = False)

1 ended.
2 ended.
3 ended.
4 ended.
5 ended.
6 ended.
7 ended.
8 ended.
9 ended.
10 ended.
11 ended.
12 ended.
13 ended.
14 ended.
15 ended.
16 ended.
17 ended.
18 ended.
19 ended.
20 ended.
21 ended.
22 ended.
23 ended.
24 ended.
25 ended.
26 ended.
27 ended.
28 ended.
29 ended.
30 ended.
31 ended.
32 ended.
33 ended.
34 ended.
35 ended.
36 ended.
37 ended.
38 ended.
39 ended.
40 ended.
41 ended.
42 ended.
43 ended.
44 ended.
45 ended.
46 ended.
47 ended.
48 ended.
49 ended.
50 ended.
51 ended.
52 ended.
53 ended.
54 ended.
55 ended.
56 ended.
57 ended.
58 ended.
59 ended.
60 ended.
61 ended.
62 ended.
63 ended.
64 ended.
65 ended.
66 ended.
67 ended.
68 ended.
69 ended.
70 ended.
71 ended.
72 ended.
73 ended.
74 ended.
75 ended.
76 ended.
77 ended.
78 ended.
79 ended.
80 ended.
81 ended.
82 ended.
83 ended.
84 ended.
85 ended.
86 ended.
87 ended.
88 ended.
89 ended.
90 ended.
91 ended.
92 ended.
93 ended.
94 ended.
95 ended.
96 ended.
97 ended.
98 ended.
99 ended.
100 ended.
101 ende

In [10]:
# train set
list_of_file_numbers = [i for i in range(1,2401)]   

# Problematic datafiles:
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(464)])
del list_of_file_numbers[list_of_file_numbers.index(464)] # 464. is problematic and will not be included in the dataset
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2189)])
del list_of_file_numbers[list_of_file_numbers.index(2189)] # 2189. is problematic and will not be included in the dataset

# Removing duplicates from the train set:
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(126)])
del list_of_file_numbers[list_of_file_numbers.index(126)] # id 395/126 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(1215)])
del list_of_file_numbers[list_of_file_numbers.index(1215)] # id 1215/1886 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(353)])
del list_of_file_numbers[list_of_file_numbers.index(353)] # id 2075/353 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(308)])
del list_of_file_numbers[list_of_file_numbers.index(308)] # id 308/2154 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(531)])
del list_of_file_numbers[list_of_file_numbers.index(531)] # id 531/1379 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2319)])
del list_of_file_numbers[list_of_file_numbers.index(2319)] # id 2319/2337 is duplicate. One removed.
print("Removing material id",list_of_file_numbers[list_of_file_numbers.index(2370)])
del list_of_file_numbers[list_of_file_numbers.index(2370)] # id 2370/2333 is duplicate. One removed.

df_general = pd.read_csv("/home/jurka/research-project/train/relaxation/general.csv")
df_egy = pd.read_csv("/home/jurka/research-project/train/relaxation/energy_relaxation.csv")
df_latt = pd.read_csv("/home/jurka/research-project/train/relaxation/lattice_vector_relaxation.csv")
df_frac = pd.read_csv("/home/jurka/research-project/train/relaxation/atoms_frac_xyz_relaxation.csv")
df_xyz = pd.read_csv("/home/jurka/research-project/train/relaxation/atoms_xyz_relaxation.csv")

for i in list_of_file_numbers:
    path = "".join(["/home/jurka/Desktop/REE/nomad_kaggle_bandfiles/train_bandfiles/just_bandfiles/", str(i), "/band.out"])
    
    df_xyz_zeroth, df_latt_zeroth, df_egy_zeroth = zeroth(path, i, df_general)
    
    #df_xyz = pd.concat([df_xyz, df_xyz_zeroth]).reset_index(drop=True)
    #df_latt = pd.concat([df_latt, df_latt_zeroth]).reset_index(drop=True)
    
    A = np.linalg.inv(lattmat(df_latt_zeroth,i,0))
    
    material = df_xyz_zeroth.loc[df_xyz_zeroth["id"]==i][df_xyz_zeroth.columns[3:].to_list()].to_numpy(dtype=float)
    
    material_frac = np.round(np.dot(material,np.transpose(A)),decimals=6)
    
    df_frac_zeroth = pd.DataFrame(material_frac, columns=["L1", "L2", "L3"])
    df_frac_zeroth = df_xyz_zeroth[df_xyz_zeroth.columns.to_list()[:3]].join(df_frac_zeroth,how="left")
    
    if len(df_latt[(df_latt["id"]==i) & (df_latt["relaxation_step_number"]==0)])==0:
        pos = df_frac.loc[df_frac["id"]==i].iloc[0].name
        df_frac = pd.concat([df_frac.iloc[:pos], df_frac_zeroth, df_frac.iloc[pos:]]).reset_index(drop=True)

        pos = df_xyz.loc[df_xyz["id"]==i].iloc[0].name
        df_xyz = pd.concat([df_xyz.iloc[:pos], df_xyz_zeroth, df_xyz.iloc[pos:]]).reset_index(drop=True)

        pos = df_latt.loc[df_latt["id"]==i].iloc[0].name
        df_latt = pd.concat([df_latt.iloc[:pos], df_latt_zeroth, df_latt.iloc[pos:]]).reset_index(drop=True)
        
        pos = df_egy.loc[df_egy["id"]==i].iloc[0].name
        df_egy = pd.concat([df_egy.iloc[:pos], df_egy_zeroth, df_egy.iloc[pos:]]).reset_index(drop=True)

df_latt.to_csv("/home/jurka/research-project/train/relaxation/with_all_zeroes/lattice.csv", index = False)
df_egy.to_csv("/home/jurka/research-project/train/relaxation/with_all_zeroes/energy.csv", index = False)
df_xyz.to_csv("/home/jurka/research-project/train/relaxation/with_all_zeroes/xyz.csv", index = False)
df_frac.to_csv("/home/jurka/research-project/train/relaxation/with_all_zeroes/frac.csv", index = False)

Removing material id 464
Removing material id 2189
Removing material id 126
Removing material id 1215
Removing material id 353
Removing material id 308
Removing material id 531
Removing material id 2319
Removing material id 2370
1 ended.
2 ended.
3 ended.
4 ended.
5 ended.
6 ended.
7 ended.
8 ended.
9 ended.
10 ended.
11 ended.
12 ended.
13 ended.
14 ended.
15 ended.
16 ended.
17 ended.
18 ended.
19 ended.
20 ended.
21 ended.
22 ended.
23 ended.
24 ended.
25 ended.
26 ended.
27 ended.
28 ended.
29 ended.
30 ended.
31 ended.
32 ended.
33 ended.
34 ended.
35 ended.
36 ended.
37 ended.
38 ended.
39 ended.
40 ended.
41 ended.
42 ended.
43 ended.
44 ended.
45 ended.
46 ended.
47 ended.
48 ended.
49 ended.
50 ended.
51 ended.
52 ended.
53 ended.
54 ended.
55 ended.
56 ended.
57 ended.
58 ended.
59 ended.
60 ended.
61 ended.
62 ended.
63 ended.
64 ended.
65 ended.
66 ended.
67 ended.
68 ended.
69 ended.
70 ended.
71 ended.
72 ended.
73 ended.
74 ended.
75 ended.
76 ended.
77 ended.
78 ended.


In [ ]:
#cat band.out | grep -Ei 'Total energy                  :|Begin self-consistency iteration #||Relaxation step number      1:|ESTIMATED overall HOMO-LUMO gap:|Total atomic forces'

In [61]:
!cat /home/jurka/Desktop/nomad_raw_files/xNP8YQvRSzWwMudTsserjg/train/str_3/band.out | grep -Ei 'Total energy                  :|ESTIMATED overall HOMO-LUMO gap:|Reading geometry description geometry.in.'

          Reading geometry description geometry.in.
  ESTIMATED overall HOMO-LUMO gap:      3.93854542 eV between HOMO at k-point 1 and LUMO at k-point 1
  | Total energy                  :      -10833.22460334 Ha     -294787.04005636 eV
  ESTIMATED overall HOMO-LUMO gap:      0.00222936 eV between HOMO at k-point 27 and LUMO at k-point 5
  | Total energy                  :      -10834.42125306 Ha     -294819.60255205 eV
  ESTIMATED overall HOMO-LUMO gap:      0.00069571 eV between HOMO at k-point 29 and LUMO at k-point 23
  | Total energy                  :      -10838.55567639 Ha     -294932.10593482 eV
  ESTIMATED overall HOMO-LUMO gap:      0.50555598 eV between HOMO at k-point 1 and LUMO at k-point 1
  | Total energy                  :      -10832.25916039 Ha     -294760.76901691 eV
  ESTIMATED overall HOMO-LUMO gap:      1.12971685 eV between HOMO at k-point 1 and LUMO at k-point 1
  | Total energy                  :      -10832.00754928 Ha     -294753.92233044 eV
  ESTIMATED ove